# Predictions

In [10]:
# Import the necessary libraries 
import pandas as pd

In [11]:
# Read in the dataframes 
books = pd.read_csv('../data/processed/books.csv')
ratings = pd.read_csv('../data/processed/ratings.csv')
top_10_SVDpp= pd.read_csv('../data/processed/top_10_SVDpp.csv', index_col=0)
top_10_KNNBaseline = pd.read_csv('../data/processed/top_10_KNNBaseline.csv', index_col=0)

In [12]:
def top_ten_prediction(user_id, top_10, books):
    if user_id not in top_10.columns: 
        print("User id", user_id, "not in the training set.")
        
        # cold_start()
        cold = pd.DataFrame(books.sort_values('bayesian_avg', ascending=False)[:10])
        cold['predicted_rating'] = cold['bayesian_avg']
        cold = cold.sort_values(['predicted_rating','bayesian_avg'], ascending=[False, False])
        #print(cold)
        return(cold)
    
    else:
        top = pd.DataFrame()
        book_list = []
        print("Top predicted books for user id", user_id)
        for book_id in top_10[user_id]:            
            to_add = books[books.book_id == eval(book_id)[0]]
            to_add['predicted_rating'] = eval(book_id)[1]
            top = pd.concat([top, to_add]) 
            top = top.sort_values(['predicted_rating','bayesian_avg'], ascending=[False, False])
        # print(top)
    return(top)

In [13]:
books[books.book_id == 8946]

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,to_read_count,tags,bayesian_avg
8934,8946,46292,46292,45423,91,9646534783,9.789647e+12,Hafez,1380.0,دیوان‎‎ [Dīvān],...,6323,286,89,116,384,872,4862,1,"to-read, favorites, currently-reading, fiction...",3.994216


In [14]:
top_10.head()

,1,2,4,8,11,15,18,22,24,25,...,38075,52199,16102,5242,12791,42977,41773,34531,7553,27329
0,"(8946, 4.647368300247821)","(7254, 5)","(8946, 4.729092406303152)","(9569, 4.794916885852385)","(8946, 4.8154324048597354)","(1380, 5)","(9569, 4.537212751942373)","(8946, 4.768484006564544)","(8946, 4.805213675292142)","(8076, 5)",...,"(8946, 4.510690945518429)","(8946, 3.9805516780300816)","(8946, 5)","(8946, 5)","(3753, 5)","(8946, 4.681517345829325)","(8451, 5)","(8946, 5)","(9076, 5)","(3753, 5)"
1,"(9076, 4.622319906117249)","(8946, 5)","(8001, 4.696491606348269)","(8946, 4.786410309161179)","(9076, 4.7073657177321815)","(8946, 5)","(9076, 4.527597939150266)","(7947, 4.719628853089255)","(9076, 4.747721981477847)","(6100, 5)",...,"(9076, 4.496120282081319)","(7876, 3.9589386722278954)","(1788, 5)","(9702, 5)","(7254, 5)","(9076, 4.674874644930957)","(9555, 5)","(4483, 5)","(8946, 4.973280607342839)","(964, 5)"
2,"(8451, 4.560119173631242)","(5754, 5)","(7947, 4.67001483795153)","(8259, 4.78033087618257)","(862, 4.63418853748627)","(5754, 5)","(8259, 4.492282439431323)","(5986, 4.6996819439069215)","(7947, 4.730108967335794)","(9076, 4.992593945838511)",...,"(8451, 4.489064451888781)","(7947, 3.9300391220704625)","(6361, 5)","(8286, 5)","(8946, 5)","(3491, 4.607862102210859)","(8442, 5)","(6902, 5)","(5788, 4.90354472085313)","(102, 5)"
3,"(6902, 4.528323712961273)","(4483, 5)","(9076, 4.6503667696399695)","(9435, 4.749911553806028)","(9569, 4.561588685860147)","(6361, 5)","(9071, 4.438731816827721)","(9076, 4.6540736875975615)","(3491, 4.657047962718113)","(7883, 4.958079437808353)",...,"(9569, 4.485519149071161)","(1577, 3.898206767474362)","(6902, 5)","(8027, 5)","(5754, 5)","(9071, 4.557911323480201)","(8001, 4.974460824941049)","(8259, 5)","(6361, 4.848181227911378)","(4, 5)"
4,"(9781, 4.5224246858345145)","(6361, 5)","(1010, 4.60782568599307)","(8001, 4.718731371018253)","(7947, 4.548981166261831)","(1010, 5)","(1010, 4.435902835807216)","(8451, 4.65290305759024)","(9071, 4.643603143968011)","(9435, 4.946655812742504)",...,"(7947, 4.471190470244539)","(4483, 3.8387687475935373)","(9076, 5)","(9076, 5)","(4483, 5)","(7947, 4.557655709436096)","(7883, 4.9721039557254105)","(9076, 5)","(7947, 4.847805227668218)","(10, 5)"


### Predict for a user in the database

In [15]:
#Predict users top_ten routes
top_books = top_ten_prediction("27329", top_10_KNNBaseline, books)
top_books.reset_index(inplace=True)
top_books[['book_id', 'title','predicted_rating', 'bayesian_avg']]

Top predicted books for user id 27329


C:\Users\cfran\AppData\Local\Temp/ipykernel_23936/3274207002.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to_add['predicted_rating'] = eval(book_id)[1]


,book_id,title,predicted_rating,bayesian_avg
0,862,"Words of Radiance (The Stormlight Archive, #2)",4.684604,4.460310
1,1788,The Calvin and Hobbes Tenth Anniversary Book,4.635462,4.333782
2,4483,It's a Magical World: A Calvin and Hobbes Coll...,4.597656,4.233393
3,780,Calvin and Hobbes,4.590825,4.433867
4,3395,The Kindly Ones (The Sandman #9),4.553374,4.227400
5,562,"The Way of Kings (The Stormlight Archive, #1)",4.547671,4.384686
6,3660,The Wake (The Sandman #10),4.545871,4.197975
7,422,"Harry Potter Boxset (Harry Potter, #1-7)",4.540397,4.479617
8,307,"The Wise Man's Fear (The Kingkiller Chronicle,...",4.539196,4.388037
9,1010,The Essential Calvin and Hobbes: A Calvin and ...,4.534223,4.409991


In [16]:
#Predict users top_ten routes
top_books = top_ten_prediction("27329", top_10_SVDpp, books)
top_books.reset_index(inplace=True)
top_books[['book_id', 'title','predicted_rating', 'bayesian_avg']]

Top predicted books for user id 27329


C:\Users\cfran\AppData\Local\Temp/ipykernel_23936/3274207002.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to_add['predicted_rating'] = eval(book_id)[1]


,book_id,title,predicted_rating,bayesian_avg
0,862,"Words of Radiance (The Stormlight Archive, #2)",4.836470,4.460310
1,4483,It's a Magical World: A Calvin and Hobbes Coll...,4.804415,4.233393
2,1618,"A Voice in the Wind (Mark of the Lion, #1)",4.726186,4.184793
3,6902,Standing for Something: 10 Neglected Virtues T...,4.714588,4.058383
4,1308,A Court of Mist and Fury (A Court of Thorns an...,4.694420,4.399902
5,1788,The Calvin and Hobbes Tenth Anniversary Book,4.674907,4.333782
6,3395,The Kindly Ones (The Sandman #9),4.667585,4.227400
7,3660,The Wake (The Sandman #10),4.667360,4.197975
8,2244,"Saga, Vol. 2 (Saga, #2)",4.658883,4.237294
9,562,"The Way of Kings (The Stormlight Archive, #1)",4.650290,4.384686


In [17]:
real_ratings = ratings[ratings.user_id == 27329]
real_ratings = pd.merge(real_ratings, books, on='book_id')
real_ratings = real_ratings.sort_values(['rating','bayesian_avg'], ascending=[False, False])
real_ratings[['book_id', 'title','rating', 'bayesian_avg']].head(10)

,book_id,title,rating,bayesian_avg
36,25,Harry Potter and the Deathly Hallows (Harry Po...,5,4.502533
38,27,Harry Potter and the Half-Blood Prince (Harry ...,5,4.422748
35,24,Harry Potter and the Goblet of Fire (Harry Pot...,5,4.411219
33,18,Harry Potter and the Prisoner of Azkaban (Harr...,5,4.400003
37,21,Harry Potter and the Order of the Phoenix (Har...,5,4.341526
32,2,Harry Potter and the Sorcerer's Stone (Harry P...,5,4.339417
87,4599,"The Crimson Crown (Seven Realms, #4)",5,4.080153
86,3677,"The Gray Wolf Throne (Seven Realms, #3)",5,4.053426
68,3765,Anne McCaffrey's Dragonflight #1,5,4.047911
85,3563,"The Exiled Queen (Seven Realms, #2)",5,4.047795


### Predict for a user not in the database

KNNBaseline and SVDpp should be the same

In [20]:
#Predict users top_ten routes
top_books = top_ten_prediction(3, top_10_SVDpp, books)
top_books.reset_index(inplace=True)
top_books[['book_id', 'title','predicted_rating', 'bayesian_avg']]

User id 3 not in the training set.


,book_id,title,predicted_rating,bayesian_avg
0,25,Harry Potter and the Deathly Hallows (Harry Po...,4.502533,4.502533
1,422,"Harry Potter Boxset (Harry Potter, #1-7)",4.479617,4.479617
2,862,"Words of Radiance (The Stormlight Archive, #2)",4.460310,4.460310
3,780,Calvin and Hobbes,4.433867,4.433867
4,27,Harry Potter and the Half-Blood Prince (Harry ...,4.422748,4.422748
5,192,The Name of the Wind (The Kingkiller Chronicle...,4.421306,4.421306
6,24,Harry Potter and the Goblet of Fire (Harry Pot...,4.411219,4.411219
7,1010,The Essential Calvin and Hobbes: A Calvin and ...,4.409991,4.409991
8,18,Harry Potter and the Prisoner of Azkaban (Harr...,4.400003,4.400003
9,1308,A Court of Mist and Fury (A Court of Thorns an...,4.399902,4.399902


In [21]:
#Predict users top_ten routes
top_books = top_ten_prediction(3, top_10_KNNBaseline, books)
top_books.reset_index(inplace=True)
top_books[['book_id', 'title','predicted_rating', 'bayesian_avg']]

User id 3 not in the training set.


,book_id,title,predicted_rating,bayesian_avg
0,25,Harry Potter and the Deathly Hallows (Harry Po...,4.502533,4.502533
1,422,"Harry Potter Boxset (Harry Potter, #1-7)",4.479617,4.479617
2,862,"Words of Radiance (The Stormlight Archive, #2)",4.460310,4.460310
3,780,Calvin and Hobbes,4.433867,4.433867
4,27,Harry Potter and the Half-Blood Prince (Harry ...,4.422748,4.422748
5,192,The Name of the Wind (The Kingkiller Chronicle...,4.421306,4.421306
6,24,Harry Potter and the Goblet of Fire (Harry Pot...,4.411219,4.411219
7,1010,The Essential Calvin and Hobbes: A Calvin and ...,4.409991,4.409991
8,18,Harry Potter and the Prisoner of Azkaban (Harr...,4.400003,4.400003
9,1308,A Court of Mist and Fury (A Court of Thorns an...,4.399902,4.399902


In [19]:
real_ratings = ratings[ratings.user_id == 3]
real_ratings = pd.merge(real_ratings, books, on='book_id')
real_ratings = real_ratings.sort_values(['rating','bayesian_avg'], ascending=[False, False])
real_ratings[['book_id', 'title','rating', 'bayesian_avg']].head(10)

,book_id,title,rating,bayesian_avg
86,4090,Celebration of Discipline: The Path to Spiritu...,5,3.956688
43,7401,The Brothers K,4,4.026598
9,4,To Kill a Mockingbird,3,4.316429
53,59,Charlotte's Web,3,4.097922
72,765,The Autobiography of Malcolm X,3,4.058373
89,15,The Diary of a Young Girl,3,4.049856
22,8492,Sailing Alone Around the Room: New and Selecte...,3,4.001476
29,2963,What's So Amazing About Grace?,3,3.989474
81,1068,The Color of Water: A Black Man's Tribute to H...,3,3.961821
79,1929,Battlefield of the Mind: Winning the Battle in...,3,3.953924
